In [66]:
#libaries
import pandas as pd
import pyreadr
import numpy as np
import os
import pickle

In [67]:
##########################################################################
# load city df and variables
##########################################################################
result = pyreadr.read_r('/data/users/laura.owen/extremes/heatwaves/HadUKGrid/dur-clim/coords/UK_top30_cities.Rda')
city_df = result['city_df']

yrs = list(range(1980, 2081, 2)) #51years
RPy = [0.5, 2, 5, 10, 20, 50, 100, 200, 500, 1000] #10RPlevels
do_dur = list(range(1, 10)) #9 durations
stens = ["01", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "15"] #12 ensembles

# Lists to store per-city results 
city_levxally_lists = {}
city_nDeqdally_lists = {}
city_th_lists = {}
city_mhwt_lists = {}

#which version of make stat to use?
make_stat = 'old' # 'old' or 'new'
if make_stat == 'old':
    main_path = '/home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs'
    output_path = "../dataframes"
elif make_stat == 'new':
    main_path = '/data/users/laura.owen/extremes/heatwaves/HadUKGrid/dur-clim/probs'
    output_path = "../dataframes/new"

In [68]:
# Initialize a global counter for files
missing_files_count = 0
found_files_count = 0

# Helper function to load .rds or return fallback array
def try_read_rds(path, fallback_shape=None):
    global missing_files_count, found_files_count
    if os.path.exists(path):
        found_files_count += 1
        result = pyreadr.read_r(path)
        return result[None]  # assume single unnamed object
    else:
        missing_files_count += 1  # Increment the counter for missing files
        if fallback_shape:
            #print(f"File not found: {path}, filling with NaNs")
            return np.full(fallback_shape, np.nan)
        else:
            raise FileNotFoundError(f"File not found and no fallback: {path}")

In [69]:
##########################################################################
# Loop over each city to get ensemble probs and ensemble mean probs to store
##########################################################################
for _, row in city_df.iterrows():
    city_name = str(row["city"]).replace(" ", "_")
    xco = str(row["lon_index"])
    yco = str(row["lat_index"])

    # create lists to store data for each ens
    levxally_list = {}
    nDeqdally_list = {}
    th_list = {}
    mhwt_list = {}
        
    ##########################################################################
    # read in all ens prob files and store
    ##########################################################################
    for s in stens:
        ens_path = f"{main_path}/{s}"
        levxally_path = f"{ens_path}/levxally_{city_name}_{xco}_{yco}_{s}.rds"
        nDeqdally_path = f"{ens_path}/nDeqdally_{city_name}_{xco}_{yco}_{s}.rds"
        th_path = f"{ens_path}/tha_{city_name}_{xco}_{yco}_{s}.rds"
        mhwt_path = f"{ens_path}/mhwt_{city_name}_{xco}_{yco}_{s}.rds"
        
        levxally_list[s] = try_read_rds(levxally_path, fallback_shape=(3, 10, 51)) #servity type (severity,mean,peakvalue), RP and years
        nDeqdally_list[s] = try_read_rds(nDeqdally_path, fallback_shape=(9, 51)) #duration and years
        th_list[s] = try_read_rds(th_path, fallback_shape=(51,))
        mhwt_list[s] = try_read_rds(mhwt_path, fallback_shape=(51,)) 

    city_levxally_lists[city_name] = levxally_list
    city_nDeqdally_lists[city_name] = nDeqdally_list
    city_th_lists[city_name] = th_list
    city_mhwt_lists[city_name] = mhwt_list

# Print the total number of missing files
print(f"Total missing files: {missing_files_count}")
print(f"Total files found: {found_files_count}")

Total missing files: 165
Total files found: 1275


In [70]:
##########################################################################
# save dictionary of th and mhwt
##########################################################################
dataframes = {}  # Dictionary to store DataFrames for each city

for city_row in city_df.itertuples():
    city = str(city_row.city).replace(" ", "_")

    th_list = city_th_lists[city]
    mhwt_list = city_mhwt_lists[city]

    data = []
    all_thresholds = []
    all_mhwts = []

    # Add ensemble members
    for ens in stens:
        df_th = th_list[ens]
        df_mhwt = mhwt_list[ens]

        thresholds = df_th.iloc[:, 0].values if isinstance(df_th, pd.DataFrame) else df_th
        mhwts = df_mhwt.iloc[:, 0].values if isinstance(df_mhwt, pd.DataFrame) else df_mhwt

        all_thresholds.append(thresholds)
        all_mhwts.append(mhwts)

        for year, threshold, mhwt in zip(yrs, thresholds, mhwts):
            data.append({
                "year": year,
                "threshold": threshold,
                "mhwt": mhwt,
                "ensemble": ens
            })

    # Compute ensemble means (handle NaNs)
    mean_thresholds = np.nanmean(all_thresholds, axis=0)
    mean_mhwts = np.nanmean(all_mhwts, axis=0)

    for year, threshold, mhwt in zip(yrs, mean_thresholds, mean_mhwts):
        data.append({
            "year": year,
            "threshold": threshold,
            "mhwt": mhwt,
            "ensemble": "mean"
        })

    # Create DataFrame for the current city
    df = pd.DataFrame(data)
    dataframes[city] = df  # Store the DataFrame in the dictionary

# Save the dataframes dictionary to a file
with open(f"{output_path}/th_mhwt_dataframes_dict.pkl", "wb") as file:
    pickle.dump(dataframes, file)

/var/tmp/ipykernel_161159/4037569345.py:36: RuntimeWarning: Mean of empty slice
  mean_thresholds = np.nanmean(all_thresholds, axis=0)
/var/tmp/ipykernel_161159/4037569345.py:37: RuntimeWarning: Mean of empty slice
  mean_mhwts = np.nanmean(all_mhwts, axis=0)


In [71]:
##########################################################################
# save dictionary of durations
##########################################################################
duration_dataframes = {}

for city_row in city_df.itertuples():
    city = str(city_row.city).replace(" ", "_")

    nDeqdally_list = city_nDeqdally_lists[city]  # dict: {ensemble: [duration x year]}
    data = []

    # Gather values for computing the mean
    all_ens_values = []

    for ens in stens:
        df_dur = nDeqdally_list[ens]
        values = df_dur.values if isinstance(df_dur, pd.DataFrame) else df_dur
        all_ens_values.append(values)

        for dur in range(1, values.shape[0] + 1):
            for year_idx, year in enumerate(yrs):
                data.append({
                    "city": city,
                    "duration": dur,
                    "year": year,
                    "nduration": values[dur - 1, year_idx],
                    "ensemble": ens
                })

    # Compute mean across ensemble members
    mean_nDeqdally = np.nanmean(all_ens_values, axis=0) # shape: [duration, year]

    for dur in range(1, mean_nDeqdally.shape[0] + 1):
        for year_idx, year in enumerate(yrs):
            data.append({
                "city": city,
                "duration": dur,
                "year": year,
                "nduration": mean_nDeqdally[dur - 1, year_idx],
                "ensemble": "mean"
            })

    # Create DataFrame for city
    df = pd.DataFrame(data)
    duration_dataframes[city] = df

# Save dictionary of duration-based data
with open(f"{output_path}/duration_dataframes_dict.pkl", "wb") as file:
    pickle.dump(duration_dataframes, file)


/var/tmp/ipykernel_161159/4089111302.py:31: RuntimeWarning: Mean of empty slice
  mean_nDeqdally = np.nanmean(all_ens_values, axis=0) # shape: [duration, year]


In [72]:
##########################################################################
# save dictionary of levxally
##########################################################################
# LEVXALLY(3,10,51) - 3 severity types, 10 RP levels, 51 years
# 1) severity 2) mean severity 3) peak value
levxally_dataframes = {}

for city_row in city_df.itertuples():
    city = str(city_row.city).replace(" ", "_")

    levxally_list = city_levxally_lists[city]  # dict: {ensemble: [severity x RP x year]}
    data = []

    # Collect all ensemble arrays
    all_levx = []

    for ens in stens:
        levx = levxally_list[ens]
        levx_values = levx.values if isinstance(levx, pd.DataFrame) else levx
        all_levx.append(levx_values)

        for sev_idx in range(levx_values.shape[0]):
            for rp_idx in range(levx_values.shape[1]):
                for year_idx, year in enumerate(yrs):
                    data.append({
                        "city": city,
                        "severity_type": sev_idx + 1,
                        "rp_level": rp_idx + 1,
                        "year": year,
                        "severity_value": levx_values[sev_idx, rp_idx, year_idx].item(),
                        "ensemble": ens
                    })

    # Compute ensemble mean
    all_levx_array = np.array(all_levx)  # shape: [ensemble, severity_type, rp_level, year]
    mean_levx = np.nanmean(all_levx_array, axis=0)  # shape: [severity_type, rp_level, year]

    # Add mean values
    for sev_idx in range(mean_levx.shape[0]):
        for rp_idx in range(mean_levx.shape[1]):
            for year_idx, year in enumerate(yrs):
                data.append({
                    "city": city,
                    "severity_type": sev_idx + 1,
                    "rp_level": rp_idx + 1,
                    "year": year,
                    "severity_value": mean_levx[sev_idx, rp_idx, year_idx].item(),
                    "ensemble": "mean"
                })

    # Create DataFrame for city
    df = pd.DataFrame(data)
    levxally_dataframes[city] = df

# Save dictionary of severity-based data
with open(f"{output_path}/levxally_dataframes_dict.pkl", "wb") as file:
    pickle.dump(levxally_dataframes, file)


/var/tmp/ipykernel_161159/3699491817.py:36: RuntimeWarning: Mean of empty slice
  mean_levx = np.nanmean(all_levx_array, axis=0)  # shape: [severity_type, rp_level, year]


In [73]:
# all_rp_levels = set()

# for df in levxally_dataframes.values():
#     all_rp_levels.update(df["rp_level"].unique())

# rp_levels_present = sorted(all_rp_levels)
# print("All RP levels present across cities:", rp_levels_present)

# RPy = [0.5, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
# rpys_present = [RPy[i - 1] for i in rp_levels_present]
# print("All RPy values present:", rpys_present)

print(levxally_dataframes["London"])

         city  severity_type  rp_level  year  severity_value ensemble
0      London              1         1  1980        0.497086       01
1      London              1         1  1982        0.562421       01
2      London              1         1  1984        0.640468       01
3      London              1         1  1986        0.715464       01
4      London              1         1  1988        0.817204       01
...       ...            ...       ...   ...             ...      ...
19885  London              3        10  2072       45.694823     mean
19886  London              3        10  2074       45.877134     mean
19887  London              3        10  2076       46.003327     mean
19888  London              3        10  2078       46.107225     mean
19889  London              3        10  2080       46.234425     mean

[19890 rows x 6 columns]
